In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np

# Load dataset
dataset = load_dataset('microsoft/wiki_qa', split='train[:1000]')

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

contexts = [item['answer'] for item in dataset]
embeddings = model.encode(contexts)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#print some example from dataset
for item in dataset:
  print(item["question"], item["answer"])

In [ ]:
import faiss

#index embeddings
embedding_dim = embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)

faiss.normalize_L2(embeddings)
index.add(np.array(embeddings))

In [ ]:
query = "how are glacier caves forme?"
query_embedding = model.encode([query])

k = 3
distances, indices = index.search(np.array(query_embedding), k)

closest_docs = [contexts[i] for i in indices[0]]


In [ ]:
closest_docs

['Ice formations in the Titlis glacier cave',
 'A glacier cave is a cave formed within the ice of a glacier .',
 'A partly submerged glacier cave on Perito Moreno Glacier .']

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

#inference
tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

combined_docs = " ".join(closest_docs)


input_text = f"summarize: {combined_docs}"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

outputs = t5_model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Response:", response)

Generated Response: glacier cave formed within the ice of a glacier.
